### Image对象转cv2(np.adarray)

```python
img = Image.open(path)
img_array = np.array(img)
```

### cv2(np.adarray)转Image对象

```python
img = cv2.imread(path)
img_Image = Image.fromarray(np.uint8(img))
```


In [ ]:
import cv2
import matplotlib.pyplot as plt

# cv2读取默认是BGR，需要转换才能正确显示
img = cv2.imread("img/obama.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)


In [ ]:
import numpy as np
from PIL import Image

fimg = np.array(Image.open("img/obama.jpg"), np.float32)
img = fimg.astype(np.uint8)
print(fimg.dtype, img.dtype)

plt.imshow(img)


imshow内部的参数类型可以分为两种

- 当输入矩阵是uint8类型的时候，此时imshow显示图像的时候，会认为输入矩阵的范围在`0-255`之间
- 当输入矩阵是double类型的时候，那么imshow会认为输入矩阵的范围在`0-1`

因此, 需要使用astype将float32转换为uint8，否则会出现一片白色([python中opencv imshow函数显示一片白色原因](https://blog.csdn.net/lyl771857509/article/details/80143134))。如果使用int8，会出现另一个问题：

```
resize.cpp:3787: error: (-215:Assertion failed) func != 0 in function 'cv::hal::resize'
```

In [ ]:
from utils import utils

fimg2 = utils.letterbox_image(img, [600, 800])
img2 = fimg2.astype(np.uint8)
plt.imshow(img2)

In [ ]:
img3 = utils.preprocess_input(fimg)
plt.imshow(img3)

In [ ]:
%%time

import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from retinaface import Retinaface
from utils import utils

retinaface = Retinaface(cuda=torch.cuda.is_available())

img = np.array(Image.open("img/twins.jpg"), np.float32)
result = retinaface.face_detect(img)
print(f'faces detected: {result.shape[0]}')
crop_imgs = []
crop_feat = []
aligned_imgs = []
aligned_feat = []

for i in range(result.shape[0]):
    # print(result[i][0:2], result[i][2:4])
    # print(result[i])
    
    b = np.array(result[i], np.int32)
    cx = b[0]
    cy = b[1] - 10
    conf = str(result[i][4])
    cropped = utils.crop_npimage(img, b)
    crop_imgs.append(cropped)
    cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
    cv2.putText(img, conf, (cx, cy), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))
    cv2.circle(img, (b[5], b[6]), 1, (0, 0, 255), 4)
    cv2.circle(img, (b[7], b[8]), 1, (0, 255, 255), 4)
    cv2.circle(img, (b[9], b[10]), 1, (255, 0, 255), 4)
    cv2.circle(img, (b[11], b[12]), 1, (0, 255, 0), 4)
    cv2.circle(img, (b[13], b[14]), 1, (255, 0, 0), 4)
    cfeat = retinaface.face_feature(cropped)
    crop_feat.append(cfeat)

    # 将landmark坐标从全图位置转换为人脸图原点偏移位置
    landmark = np.reshape(b[5:],(5,2)) - np.array([int(b[0]),int(b[1])])
    # 做人脸对齐
    aligned, _ = utils.Alignment_1(cropped, landmark)
    aligned_imgs.append(aligned)
    afeat = retinaface.face_feature(aligned)
    aligned_feat.append(afeat)
    # print(f'shape type:{type(cfeat)} c:{cfeat.shape} cl:{len(cfeat)} a:{afeat.shape}')
    print(f'cfeat & afeat distance: {utils.face_distance(cfeat, afeat, axis=0)}')

img = img.astype(np.uint8)

nrow = 2
ncol = 8
# fig, axs = plt.subplots(3, 1, constrained_layout=True, figsize=(20, 20))
fig = plt.figure(3, constrained_layout=True, figsize=(20, 20))
sf1, sf2, sf3 = fig.subfigures(3, 1)
sf1.suptitle('output')
sf2.suptitle('cropped')
sf3.suptitle('aligned')

plt_out = sf1.subplots(1, 1)
plt_out.imshow(img)
plt_crop = sf2.subplots(nrow, ncol)
plt_align = sf3.subplots(nrow, ncol)

# fig = plt.figure(figsize=[10.0, 10.0])
# fig.add_subplot(i,2,1)
# plt.imshow(img)

for i in range(len(crop_imgs)):
    if (i < nrow * ncol):
        plt_crop[i // ncol, i % ncol].imshow(crop_imgs[i].astype(np.uint8))
        plt_align[i // ncol, i % ncol].imshow(aligned_imgs[i].astype(np.uint8))

plt.show()

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(20, 20))
subfig = fig.subfigures(2, 1)
axs = subfig[0].subplots(nrow, ncol)

for i in range(result.shape[0]):
    r = result[i]
    # 将landmark坐标从全图位置转换为人脸图原点偏移位置
    landmark = np.reshape(r[5:],(5,2)) - np.array([int(r[0]),int(r[1])])
    # 做人脸对齐
    aligned, _ = utils.Alignment_1(crop_imgs[i], landmark)
    axs[i // ncol, i % ncol].imshow(aligned.astype(np.uint8))


In [ ]:
%%time
%matplotlib inline
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
from retinaface import Retinaface
from utils import utils

retinaface = Retinaface(cuda=torch.cuda.is_available())

def extract_feature(image_file):
    img = np.array(Image.open(image_file), np.float32)
    img = img[:,:,:3]

    result = retinaface.face_detect(img)
    if len(result) > 1:
        raise ('more than one face was detected')

    result = result[0]
    # print(result)
    bbox = result[0:4].astype(np.uint)
    conf = result[4]
    lm = result[5:].astype(np.uint).reshape(5, 2)

    # print(bbox)
    # print(conf)
    # print(lm)

    cimg = utils.crop_npimage(img, bbox)
    # cv2.imwrite("c1.jpg", cv2.cvtColor(cimg, cv2.COLOR_RGB2BGR))

    # 将landmark坐标从全图位置转换为人脸图原点偏移位置
    lm = lm - np.array([int(bbox[0]),int(bbox[1])])
    align, _ = utils.Alignment_1(cimg, lm)
    # cv2.imwrite("c2.jpg", cv2.cvtColor(align, cv2.COLOR_RGB2BGR))

    feat0 = retinaface.face_feature(cimg)

    feat = retinaface.face_feature(align)
    return cimg, align, feat

origin = '../dataset/manual/face/lyf1.jpg'
comparison = [
    '../dataset/manual/face/jacky1.png', 
    '../dataset/manual/face/jacky2.png', 
    '../dataset/manual/face/jacky3.png',
    '../dataset/manual/face/jacky4.png',
    '../dataset/manual/face/jacky5.png',
    '../dataset/manual/face/sa1.jpg', 
    '../dataset/manual/face/sa2.jpg', 
    '../dataset/manual/face/sa3.jpg', 
    '../dataset/manual/face/gillian2.jpg', 
    '../dataset/manual/face/gillian3.jpg', 
    '../dataset/manual/face/gillian4.jpg', 
    '../dataset/manual/face/gillian5.jpg', 
    '../dataset/manual/face/gillian6.jpg', 
    '../dataset/manual/face/gillian7.jpg', 
    '../dataset/manual/face/lyf1.jpg', 
    '../dataset/manual/face/lyf2.jpg', 
    '../dataset/manual/face/lyf3.jpg', 
    '../dataset/manual/face/lyf4.jpg', 
    '../dataset/manual/face/lyf5.jpg', 
    '../dataset/manual/face/ycl1.jpg', 
]

fig = plt.figure(2, constrained_layout=True, figsize=(20, 20))
sf1, sf2 = fig.subfigures(2, 1)
sf1.suptitle('input')
sf2.suptitle('comparison')

cimg, aimg, origin_feat = extract_feature(origin)

ax = sf1.subplots(1, 1)
ax.imshow(aimg.astype(np.uint8))

ax = sf2.subplots(len(comparison) // 4 + 1, 4)
for i, f in enumerate(comparison):
    ci, ai, feat = extract_feature(f)
    dist = utils.face_distance(origin_feat, feat, 0)
    ci = ci.astype(np.uint8)
    ai = ai.astype(np.uint8)
    ax[i // 4][i % 4].set_title(str(round(dist, 3)))
    ax[i // 4][i % 4].imshow(ai)
plt.show()
